In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import csv
import re

chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument("--blink-settings=imagesEnabled=false") # disabling images to improve run time 
chrome_options.add_argument('--headless')  # Enable headless mode
chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration in headless mode (optional)
driver = webdriver.Chrome(options=chrome_options)



fields = ["Link", "Title", "Year", "Rating"] # creating data columns
filename = "test.csv" # new file
# creating lists 
rows = []
links = []
titles_temp = [] #temporary container for movie info 
titles = []
year = []
rating = []
errors = []
synopsis = []

for i in range(1,1):
    url = f"https://letterboxd.com/films/popular/page/{i}/"
    driver.get(url)
    try:
        #movies = driver.find_elements(By.XPATH, "//*[@id='films-browser-list-container']/ul/li") # container of all movies on page
        # i put by css selector because it is supposed to be faster
        movies = driver.find_elements(By.CSS_SELECTOR, "#films-browser-list-container ul li")
        for movie in movies: # iterate through movies 
            movie_info = movie.find_element(By.TAG_NAME, "a").get_attribute('data-original-title')
            if movie_info.find("(") == -1 or movie_info[-1].isdigit() == False:
                continue
            titles_temp.append(movie_info) # movie info
            link = movie.find_element(By.TAG_NAME, "a").get_attribute('href')
            links.append(link) # link

    except: #print page number of where error occurs
        errors.append(i)
        continue
for i in errors:
    url = f"https://letterboxd.com/films/popular/page/{i}/"
    driver.get(url)
    try:
        #movies = driver.find_elements(By.XPATH, "//*[@id='films-browser-list-container']/ul/li") # container of all movies on page
        # i put by css selector because it is supposed to be faster
        movies = driver.find_elements(By.CSS_SELECTOR, "#films-browser-list-container ul li")
        for movie in movies: # iterate through movies 
            
            movie_info = movie.find_element(By.TAG_NAME, "a").get_attribute('data-original-title')
            if movie_info.find("(") == -1 or movie_info[-1].isdigit() == False:
                continue
            titles_temp.append(movie_info) # movie info
            link = movie.find_element(By.TAG_NAME, "a").get_attribute('href')
            links.append(link) # link
            
    except: #print page number of where error occurs
        print("page: "+str(i))
        continue

for film in titles_temp: #iterate through movie info for each one
    try:
    # append separate features of the movie info 
        titles.append(film[:-11].strip()) 
        year.append(film[-10:-6].strip())
        rating.append(film[-4:].strip())
    # print movie info when there's an error to track it 
    except IndexError:
        print(new_list)
        print(film)
        continue
for i in range(len(titles)-1): 
    try:
        rows.append([links[i], titles[i], year[i], rating[i]]) # create rows of csv 
    except IndexError:
        print(i) # print index where error occurs 
with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields) 
    csvwriter.writerows(rows)

driver.quit()
print("Done")
# done message is because you dont see the webscraping in action in headless mode
# 500 pages should take rougly 20ish minutes 

Error getting version of chromedriver 116. Retrying with chromedriver 115 (attempt 1/5)
Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 2/5)


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 116.0.5845.110 with binary path /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
Stacktrace:
0   chromedriver                        0x00000001004a7f48 chromedriver + 4226888
1   chromedriver                        0x00000001004a04f4 chromedriver + 4195572
2   chromedriver                        0x00000001000e4d68 chromedriver + 281960
3   chromedriver                        0x0000000100110bac chromedriver + 461740
4   chromedriver                        0x000000010010cfd8 chromedriver + 446424
5   chromedriver                        0x000000010010a150 chromedriver + 434512
6   chromedriver                        0x000000010014793c chromedriver + 686396
7   chromedriver                        0x0000000100147164 chromedriver + 684388
8   chromedriver                        0x0000000100113f1c chromedriver + 474908
9   chromedriver 

In [1]:
from bs4 import BeautifulSoup
import lxml
import requests
import requests_cache
import csv
import re
import pandas as pd
import cchardet as chardet
from multiprocessing import Pool

requests_cache.install_cache('my_cache', expire_after=86400)
requests_session = requests.Session()

df = pd.read_csv('Letterboxd_scrape_first500.csv')
synopsis = []
for index,link in enumerate(df["Link"]):
    try:
        
        source = requests_session.get(link)
        soup = BeautifulSoup(source.content, 'lxml')
        main = soup.find("div", class_ = "content-wrap")
        main = main.find("div", class_ = "review body-text -prose -hero prettify")
        synop = main.find("div", class_ = 'truncate').p.text
        synopsis.append(synop)
    except:
        synopsis.append("")
        continue
df['Synopsis'] = pd.Series(synopsis)
df.to_csv('Letterboxd_scrape_first500_v2.csv')  
print("Done")

Done
